In [ ]:
from google.colab import drive
drive.mount('/content/gdirve')

Drive already mounted at /content/gdirve; to attempt to forcibly remount, call drive.mount("/content/gdirve", force_remount=True).


In [ ]:
pip install konlpy

In [ ]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import json
import re
from bs4 import BeautifulSoup
import warnings
import numpy as np
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import jpype
from pathlib import Path
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer


warnings.filterwarnings('ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
!apt-get install fonts-nanum*
!apt-get install fontconfig
!fc-cache -fv
!cp /usr/share/fonts/truetype/nanum/Nanum* /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/
!rm -rf /content/.cache/matplotlib/*

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'fonts-nanum-eco' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-gothic-light' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-coding' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-extra' for glob 'fonts-nanum*'
fonts-nanum is already the newest version (20170925-1).
fonts-nanum-coding is already the newest version (2.5-1).
fonts-nanum-eco is already the newest version (1.000-6).
fonts-nanum-extra is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fontconfig is already the newest version (2.12.6-0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetyp

In [ ]:
import os
os.chdir('/content/gdirve/MyDrive/data')

## 어플별 리뷰데이터 통합

In [ ]:
data_g1 = pd.read_csv("삼성증권.csv")
data_a1 = pd.read_csv('a삼성증권.csv')

data_g2 = pd.read_csv("미래에셋.csv")
data_a2 = pd.read_csv('a미래에셋.csv')

data_g3 = pd.read_csv("나무.csv")
data_a3 = pd.read_csv('a나무.csv')

data_g4 = pd.read_csv("영웅문.csv")
data_a4 = pd.read_csv('a영웅문.csv')

data_g5 = pd.read_csv("한투.csv")
data_a5 = pd.read_csv('a한투.csv')

data_g6 = pd.read_csv("KB.csv")
data_a6 = pd.read_csv('aKB.csv')

data_g = pd.concat([data_g1, data_g2, data_g3, data_g4, data_g5, data_g6], axis=0)
data_a = pd.concat([data_a1, data_a2, data_a3, data_a4, data_a5, data_a6], axis=0)

In [ ]:
print(data_g.head())
print(data_a.head())

        name  ...  developer_comment
0       종합선물  ...                NaN
1  John Yeon  ...                NaN
2      Lim S  ...                NaN
3        김주현  ...                NaN
4         잘가  ...                NaN

[5 rows x 6 columns]
                USER  ...                                             REVIEW
0                수녀니  ...  휴대폰 바꿔서 다시 로그인하려고 계속 시도중인데 공동인증서 등록이 마지막 단계에서 ...
1  Alex sangbeom Kim  ...          금융감독원 앱 설치 없이 페이스 아이디 로그인이 가능하도록 개선 바랍니다.
2              동배루스시  ...  알뜰폰 안되고 은행인증서 안되고 ㅋㅋㅋㅋ\n왜 갑질이세요\n수많은 증권사 계좌개설했...
3                쮜블리  ...                  저뿐만아니라 친구들도 토론방에만 들어가면 멈춰요 해결해주세요
4        yxxnsxhxxxx  ...                  토론방 들어갈때마다 답답해 죽겠어요 진짜 제발 업데이트 제발

[5 rows x 6 columns]


In [ ]:
data_g = data_g[['ratings','comment']]
data_a = data_a[['STAR','REVIEW']]

In [ ]:
data_a.columns=['ratings','comment']
data = pd.concat([data_g, data_a], axis=0)
data.head()

,ratings,comment
0,4,증권mts중에 최고입니다. 그러나 일부사항 개선해주시갈 바랍니다. 1.간편이체 화면...
1,1,1년 넘게 쓰는데 앱 켜면 매일 뜨는 자사 광고가 너무 피로하게 만드네요. 어차피 ...
2,1,로그인 로딩만 뜨고 아얘 안됨. 발열 심함 1. 휴대폰 산지 한달도 안되어서 기기문...
3,3,자동감시주문 매도에서 수량을 퍼센트로 설정할 수 있도록 해주세요. 감시 금액도 이하...
4,2,몇일전부터 발열이 너무심하고 배터리가 굉장히 빨리 소모됩니다. 예전에는 안그랬는데 ...


In [ ]:
high_rate_review = data[data['ratings'] >= 4][['comment', 'ratings']]
low_rate_review = data[data['ratings'] <= 2][['comment', 'ratings']]

In [ ]:
high_rate_review['ratings'] = 1
low_rate_review['ratings'] = 0

In [ ]:
print(high_rate_review[:10])
print(low_rate_review[:10])

                                              comment  ratings
0   증권mts중에 최고입니다. 그러나 일부사항 개선해주시갈 바랍니다. 1.간편이체 화면...        1
10  안녕하세요. 잘 사용하고 있습니다 그런데. 주식 잔고 화면의 컬럼을 변경할수 있으면...        1
19  타 증권사들것 보다 쓰기 좋아서 정착했습니다. 그런데 폰 교체하면 관심종목 모아놓은...        1
23  뭐야 평점을 낮게해야 보는거야? 기준선에 "평균매입가" 추가해주세요.// 다 좋은데...        1
27  증권사가 상당히 많이 보유하고 있는데 기존 증권사 업데이트 불편함으로 삼성증권을 주...        1
29  앱을열면 업데이트중이라고 써있고 하루종일 멈춰있어요. 전원 끄고. 앱제거 하고.. ...        1
32  어플은 그럭저럭 쓸만하다. 단점은 매수나 매도시 주식수량 정정할때 불편하다. 다른 ...        1
34  국내주식은 잔고에서 평가금액을 누르면 평가액이 많은 순으로 나열이 되는데 해외주식은...        1
37  증권어플중 삼성증권이 가장보기 편해서 잘사용하고 있습니다. 다만 차트에 매수평균가 ...        1
39  잘 쓰고 있습니다. 뭐 깔끔하고 좋은데 제가 쓰는 용도에 건의하고 싶은게 있습니다....        1
                                              comment  ratings
1   1년 넘게 쓰는데 앱 켜면 매일 뜨는 자사 광고가 너무 피로하게 만드네요. 어차피 ...        0
2   로그인 로딩만 뜨고 아얘 안됨. 발열 심함 1. 휴대폰 산지 한달도 안되어서 기기문...        0
4   몇일전부터 발열이 너무심하고 배터리가 굉장히 빨리 소모됩니다. 예전에는 안그랬는데 ...        0
5   최근조회항목 초기화되는것때문에 증권사 바꿔야겠다고 생각중입니다. 여기서 사람들이 엄...  

In [ ]:
review_data = pd.concat([high_rate_review, low_rate_review], axis=0)
review_data.head()

,comment,ratings
0,증권mts중에 최고입니다. 그러나 일부사항 개선해주시갈 바랍니다. 1.간편이체 화면...,1
10,안녕하세요. 잘 사용하고 있습니다 그런데. 주식 잔고 화면의 컬럼을 변경할수 있으면...,1
19,타 증권사들것 보다 쓰기 좋아서 정착했습니다. 그런데 폰 교체하면 관심종목 모아놓은...,1
23,"뭐야 평점을 낮게해야 보는거야? 기준선에 ""평균매입가"" 추가해주세요.// 다 좋은데...",1
27,증권사가 상당히 많이 보유하고 있는데 기존 증권사 업데이트 불편함으로 삼성증권을 주...,1


### 결측치 제거

In [ ]:
review_data.isnull().sum()

comment    21
ratings     0
dtype: int64

In [ ]:
review_data.dropna(inplace=True)

In [ ]:
review_data.isnull().sum()

comment    0
ratings    0
dtype: int64

In [ ]:
print(len(review_data))

45240


# RNN

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 데이터 변환

In [ ]:
class APP_sequence(Dataset):
    def __init__(self, data, max_seq_len):
        self.max_seq_len = max_seq_len
        df = data
        # Print the first ten lines
        print(df.head(10))
        # Data shape
        print(df.shape)

        # Use CountVectorizer()
        vectorizer = CountVectorizer(min_df=0.001, ngram_range=(1,2))
        vectorizer.fit(df.comment.tolist())
        
        self.vocab = vectorizer.vocabulary_
        # Add padding token
        self.vocab['<PAD>'] = max(self.vocab.values()) + 1

        # Create a tokenizer instance that lets you extract the tokenizing step from the pipeline wrapped in CountVectorizer  
        tokenizer = vectorizer.build_tokenizer()
        self.encode = lambda x: [self.vocab[token] for token in tokenizer(x)
                                 if token in self.vocab]
        self.pad = lambda x: x + (max_seq_len - len(x)) * [self.vocab['<PAD>']]
        
        # Get sequence from each row and prepare it to have length max_seq_len
        sequences = [self.encode(sequence)[:max_seq_len] for sequence in df.comment.tolist()]
        sequences, self.labels = zip(*[(sequence, label) for sequence, label
                                    in zip(sequences, df.ratings.tolist()) if sequence])
        self.sequences = [self.pad(sequence) for sequence in sequences]

    def __getitem__(self, i):
        assert len(self.sequences[i]) == self.max_seq_len
        return self.sequences[i], self.labels[i]
    
    def __len__(self):
        return len(self.sequences)

In [ ]:
dataset = APP_sequence(data=review_data, max_seq_len=128)

                                              comment  ratings
0   증권mts중에 최고입니다. 그러나 일부사항 개선해주시갈 바랍니다. 1.간편이체 화면...        1
10  안녕하세요. 잘 사용하고 있습니다 그런데. 주식 잔고 화면의 컬럼을 변경할수 있으면...        1
19  타 증권사들것 보다 쓰기 좋아서 정착했습니다. 그런데 폰 교체하면 관심종목 모아놓은...        1
23  뭐야 평점을 낮게해야 보는거야? 기준선에 "평균매입가" 추가해주세요.// 다 좋은데...        1
27  증권사가 상당히 많이 보유하고 있는데 기존 증권사 업데이트 불편함으로 삼성증권을 주...        1
29  앱을열면 업데이트중이라고 써있고 하루종일 멈춰있어요. 전원 끄고. 앱제거 하고.. ...        1
32  어플은 그럭저럭 쓸만하다. 단점은 매수나 매도시 주식수량 정정할때 불편하다. 다른 ...        1
34  국내주식은 잔고에서 평가금액을 누르면 평가액이 많은 순으로 나열이 되는데 해외주식은...        1
37  증권어플중 삼성증권이 가장보기 편해서 잘사용하고 있습니다. 다만 차트에 매수평균가 ...        1
39  잘 쓰고 있습니다. 뭐 깔끔하고 좋은데 제가 쓰는 용도에 건의하고 싶은게 있습니다....        1
(45240, 2)


### Batch Collate

In [ ]:
def collate(batch):
    inputs = torch.LongTensor([item[0] for item in batch])
    target = torch.FloatTensor([item[1] for item in batch])
    return inputs, target

batch_size = 2048
train_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate)

### RNN모델 구축(GRU 레이어 사용)

In [ ]:
class RNN_sentiment(nn.Module):
    def __init__(
        self,
        vocab_size,
        batch_size,
        embedding_dimension=100,
        hidden_size=128, 
        num_layers=3,
        device='cpu',
    ):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size # number of features in the hidden state h
        self.device = device
        self.batch_size = batch_size
        
        self.encoder = nn.Embedding(vocab_size, embedding_dimension)
        self.gru = nn.GRU(
            embedding_dimension,
            hidden_size,
            num_layers=num_layers,
            batch_first=True, # provide input and output tensors as (batch, seq, hidden_size)
        )
        self.decoder = nn.Linear(hidden_size, 1)
        
    def init_hidden(self):
        return torch.randn(self.num_layers, self.batch_size, self.hidden_size).to(self.device)
            
    def forward(self, inputs):
        # Avoid breaking if the last batch has a different size
        batch_size = inputs.size(0)
        if batch_size != self.batch_size:
            self.batch_size = batch_size
            
        encoded = self.encoder(inputs)
        output, hidden = self.gru(encoded, self.init_hidden())
        # output[batch_size, seq, hidden_size]
        # Take all from batch and seq but only output from last layer
        output = self.decoder(output[:, :, -1]).squeeze()
        return output

In [ ]:
model = RNN_sentiment(
    hidden_size=128,
    vocab_size=len(dataset.vocab),
    device=device,
    batch_size=batch_size,
)
model = model.to(device)
model

RNN_sentiment(
  (encoder): Embedding(1098, 100)
  (gru): GRU(100, 128, num_layers=3, batch_first=True)
  (decoder): Linear(in_features=128, out_features=1, bias=True)
)

### Train

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

from tqdm.notebook import tqdm, tqdm_notebook

EPOCHS = 60

model.train()
train_losses = []
for epoch in range(EPOCHS):
    progress_bar = tqdm_notebook(train_loader, leave=False)
    losses = []
    total = 0
    for inputs, target in progress_bar:
        inputs, target = inputs.to(device), target.to(device)

        model.zero_grad()

        output = model(inputs)
    
        loss = criterion(output, target)
        
        loss.backward()
              
        nn.utils.clip_grad_norm_(model.parameters(), 3)

        optimizer.step()
        
        progress_bar.set_description(f'Epoch {epoch+1} Loss: {loss.item():.4f}')
        
        losses.append(loss.item())
        total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)

    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

  0%|          | 0/19 [00:00<?, ?it/s]

Epoch #1	Train Loss: 0.465


  0%|          | 0/19 [00:00<?, ?it/s]

Epoch #2	Train Loss: 0.352


  0%|          | 0/19 [00:00<?, ?it/s]

...

Epoch #58	Train Loss: 0.213


  0%|          | 0/19 [00:00<?, ?it/s]

Epoch #59	Train Loss: 0.213


  0%|          | 0/19 [00:00<?, ?it/s]

Epoch #60	Train Loss: 0.210


### 모델 평가 및 예측

In [ ]:
def predict_sentiment(text):
    model.eval()
    with torch.no_grad():
        # Encode, pad and convert to tensor
        test_vector = torch.LongTensor([dataset.pad(dataset.encode(text))]).to(device)
        
        output = model(test_vector)
        prediction = torch.sigmoid(output).item()

        if prediction > 0.5:
            print(f'{prediction:0.3f}: Positive sentiment')
        else:
            print(f'{prediction:0.3f}: Negative sentiment')

In [ ]:
# Negative test1: 별점 1점

test_txt = """
실현손익에 기간 당월또는 오늘로 바꾸는거 어려운거 아닐텐데...수정이안되는건 개발자들의 게으름이 아니라 설계자의 고집일거란 생각이 드네요. 매도시 없어진 1/2 3/1수량 지정도...ㅜㅜ 같은 개발자 입장에서 코딩 몇줄 이면 다해결되는 일인거 아는데... 개발자가 실력이안되거나 설계자가 어리석은 고집쟁이거나...둘중 하나겠죠...안타깝네요
"""
predict_sentiment(test_txt)

0.025: Negative sentiment


In [ ]:
# Negative test 2: 별점 2점

test_txt = """
업데이트 하고 더욱 더 불편해졌어요;; 예전 업데이트도 짜증났지만 관심종목 보는 것도 일일이 다시 봐야해서 귀찮아졌고 심지어 어떤 불가사의한 알고리즘으로 차트가 알아서 커지거나 작아지던데 이건 도대체 누구 머리에서 나온 아이디어인지 모르겠습니다. 누군진 모르겠지만 어플 디자이너 당장 모가지 자르고, 제발 이 눈아픈 새하얀 화면을 나이트모드화 해서 어둡게도 볼 수 있게좀 해주세요. 내가 지금까지 지불한 거래 수수료가 어떤 정체불명의 무능한 디자이너 지갑으로 들어갔다고 생각하면 kb증권으로 다 빼고만 싶어지는군요. 다른 증권사 어플이라도 좀 본받으세요
"""
predict_sentiment(test_txt)

0.016: Negative sentiment


In [ ]:
# Positive test: 별점 5점

test_txt = """
투자에편리한앱개발해주셔서감사합니다잘사용하고있습니다~~그런데요며칠차트설정이계속초기화되는오류가있어앱을재차다운받는번거로움이있었습니다뭐가문제인지모르겠네요?그리고 보조지표선굵기를아주아주굵게설정할수있었으면좋겠습니다~~감사합니다 ~~수고하세여!
"""
predict_sentiment(test_txt)

0.980: Positive sentiment


In [ ]:
# Tricky test: 별점 3점

test_txt = """
1. 가로종합주문 창이 없어졌나요? 휴대폰 주문시 챠트와 주문창을 동시에 띄어 좀더 적시에 주문이 가능했어요. 착오로 삐졌다면 반드시 다시 가로종합주문 창을 넣어주세요.(다른 기능에 포함되어 있다면 화면 보는법 알려주세요). 2. 기존대로 "주식현재가->관심그룹선택->종목등록" 토록 기존기능 복원 강력히 요구합니다. 3. 챠트에서 각종 보조지표를 활용하고 있는데 캔들 터치시 화면에 지표값 전체가 안보여요. 제일 중요한 시가,고가, 저가, 종가가 안보여 난감합니다. 지표값 전부 보이게 해주던지, 아니면 시가, 고가, 저가, 종가, 거래량이 맨아래쪽에 위치시켜 볼 수 있도록 개선요구합니다.
"""
predict_sentiment(test_txt)

0.065: Negative sentiment
